# Random Forest Classification 

## Load Dataset

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import sys, pathlib
sys.path.append("..")
import pandas as pd
from Datasets.MultiModalClassificationDataset import MultiModalClassificationConfig, MultiModalClassificationDataset
config = MultiModalClassificationConfig()
multi_modal_classification_dataset = MultiModalClassificationDataset(excel_file=config.DEFAULT_EXCEL_PATH, sheet_names=["CFP"])
columns = ["img_path", "diagnosis"]
multi_modal_classification_df = pd.DataFrame(multi_modal_classification_dataset.samples, columns=columns)
print(multi_modal_classification_df.head())
print(multi_modal_classification_df.describe())

ignore FFA modal
ignore ultrasound modal
ignore OCT modal
ignore slitlamp modal
                                            img_path             diagnosis
0  /home/hongyu/Visual-RAG-LLaVA-Med/data/Multimo...  diabetic retinopathy
1  /home/hongyu/Visual-RAG-LLaVA-Med/data/Multimo...  diabetic retinopathy
2  /home/hongyu/Visual-RAG-LLaVA-Med/data/Multimo...  diabetic retinopathy
3  /home/hongyu/Visual-RAG-LLaVA-Med/data/Multimo...  diabetic retinopathy
4  /home/hongyu/Visual-RAG-LLaVA-Med/data/Multimo...  diabetic retinopathy
                                                 img_path  \
count                                                 208   
unique                                                208   
top     /home/hongyu/Visual-RAG-LLaVA-Med/data/Multimo...   
freq                                                    1   

                   diagnosis  
count                    208  
unique                    13  
top     diabetic retinopathy  
freq                      16  


## Build Train Test Sets (only Multi Modal VQA)

In [10]:
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

def load_image(img_path, size=(1024, 1024)):
    """Load an image from a path and resize it."""
    img = Image.open(img_path).convert('L')  # 转换为灰度图
    img = img.resize(size)  # 调整大小
    return np.array(img).flatten()  # 将图片展平为一维数组

multi_modal_classification_df['diagnosis'] = multi_modal_classification_df['diagnosis'].astype('category')
multi_modal_classification_df['features'] = multi_modal_classification_df['img_path'].apply(load_image)

X = np.vstack(multi_modal_classification_df['features'].values)
y = multi_modal_classification_df['diagnosis']

# 使用train_test_split进行数据集的划分，test_size=0.2表示测试集占20%
# stratify=y参数用于保证训练集和测试集中各类别样本的比例与原始数据集中相同
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



## Random Forest

In [11]:
from sklearn.ensemble import RandomForestClassifier
# 创建随机森林分类器实例
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# 训练模型
clf.fit(X_train, y_train)

# 测试模型准确率
accuracy = clf.score(X_test, y_test)
print(f"模型准确率: {accuracy * 100:.2f}%")

RandomForestClassifier(random_state=42)

模型准确率: 14.29%


experiments shows increase the size of image can improve the accuracy